In [4]:
import os
import numpy as np
import cv2 as cv2
import random
from random import shuffle
from tqdm import tqdm

In [6]:
#PATHS
PICASSO_TRAIN = "/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/Training/Picasso_training"
VANGOGH_TRAIN = "/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/Training/vangogh_training"
PICASSO_TEST = "/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/Testing/Picasso_testing"
VANGOGH_TEST = "/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/Testing/vangogh_testing"
#CONSTANTS
IMAGE_SIZE = 50
LR = 1e-3
MODEL_NAME = "PXVG--{}".format('conv-demo')

In [109]:
print(os.listdir(PICASSO_TRAIN)[0])

PC.1004.jpg


In [110]:
#GETTING THE LABEL FOR ALL OF THE TRAINING DATA
def getlabel(img):
    label = img.split('.')[-3]
    if label == 'PC':
        return np.array([1,0])
    elif label == 'VG':
        return np.array([0,1])

In [111]:
#PREPARING THE IMAGES
training_data = []
def getimage():
    for i in tqdm(os.listdir(PICASSO_TRAIN)):
        current_path = os.path.join(PICASSO_TRAIN,i)
        image = cv2.imread(current_path,cv2.IMREAD_GRAYSCALE)
        try:
            new_image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
        except:
            return ("invalid",i)
        lbl = getlabel(i)
        training_data.append([np.array(new_image),np.array(lbl)])
    #vangogh
    for i in tqdm(os.listdir(VANGOGH_TRAIN)):
        current_path = os.path.join(VANGOGH_TRAIN,i)
        image = cv2.imread(current_path,cv2.IMREAD_GRAYSCALE)
        try:
            new_image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
        except:
            return ("invalid",i)
        lbl = getlabel(i)
        training_data.append([np.array(new_image),np.array(lbl)])
    np.save('train.npy',training_data)
        
        

In [112]:
training_data = []
getimage()

100%|██████████| 1088/1088 [00:08<00:00, 129.86it/s]


In [13]:
full_data = np.load('train.npy',allow_pickle = True)

In [14]:
f = full_data[1]
print(f)

[array([[44, 44, 44, ..., 57, 54, 47],
       [43, 43, 45, ..., 63, 43, 34],
       [51, 51, 42, ..., 60, 34, 33],
       ...,
       [69, 73, 76, ..., 36, 36, 36],
       [70, 71, 73, ..., 36, 36, 36],
       [70, 62, 68, ..., 35, 36, 36]], dtype=uint8)
 array([1, 0])]


In [15]:
train = full_data[:-500]
test = full_data[-500:]

In [16]:
print(train.shape)

(1802, 2)


In [17]:
print(test.shape)

(500, 2)


In [18]:
X = np.array([i[0] for i in train]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
test_y = [i[1] for i in test]

In [19]:
import tensorflow
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [22]:
tensorflow.reset_default_graph()

In [11]:
#CREATING THE MODEL
convnet = input_data(shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

W1105 18:54:24.647781 4337653184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1105 18:54:24.653857 4337653184 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1105 18:54:24.654756 4337653184 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for u

In [23]:
model.fit({'input': X}, {'targets': Y}, n_epoch=20, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=100, show_metric=True, run_id=MODEL_NAME)

Exception: The collection `tf.GraphKeys.INPUTS` is empty! Cannot retrieve placeholder. In case placeholder was defined outside TFLearn `input_data` layer, please add it to that collection.

In [ ]:
#LOAD IN THE IMAGE FOR PREDICTION
t_image = cv2.imread(os.path.join(PICASSO_TEST,'80.jpg'))
try:
    t_image = cv2.resize(t_image,(IMAGE_SIZE,IMAGE_SIZE))
except:
    print("broken")
t_image = np.array(t_image).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
prediction = model.predict(t_image)

In [76]:
print(prediction[2])

[0.7979768  0.20202322]


In [26]:
def predict(image):
    t_image = cv2.imread(image)
    try:
        t_image = cv2.resize(t_image,(IMAGE_SIZE,IMAGE_SIZE))
    except:
        print("broken")
    t_image = np.array(t_image).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
    prediction = model.predict(t_image)
    if prediction[2][0]>0.5:
        print('picasso')
        print(prediction[2][0])
    else:
        print('van gogh')
        print(prediction[2][1])

In [28]:
predict('/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/ProgramFiles/FlaskApp/static/picasso_test.jpg')

picasso
0.53854054


In [29]:
predict('/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/testing/Picasso_testing/82.jpg')

picasso
0.52189505


In [30]:
model.save('pXvg.tflearn')

W1105 19:12:47.061860 4337653184 meta_graph.py:449] Issue encountered when serializing data_preprocessing.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'
W1105 19:12:47.063570 4337653184 meta_graph.py:449] Issue encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'
W1105 19:12:47.067760 4337653184 meta_graph.py:449] Issue encountered when serializing summary_tags.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'dict' object has no attribute 'name'


In [131]:
#CREATING THE MODEL
convnet = input_data(shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')
model.load('/Users/athreya/Desktop/GAN_Exploration/PicassoArtClassifier/ProgramFiles/pXvg.tflearn')

NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Accuracy/Mean/moving_avg_1 not found in checkpoint
	 [[node save_53/RestoreV2 (defined at /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/helpers/trainer.py:147) ]]

Original stack trace for 'save_53/RestoreV2':
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib

In [126]:
model.fit({'input': X}, {'targets': Y}, n_epoch=20, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=100, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: PXVG--conv-demo
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 16218
Validation samples: 4500
--


RuntimeError: Attempted to use a closed Session.